In [1]:
import pandas as pd

df = pd.read_csv("/kaggle/input/labeled-cmt2/labeled_cmt - Sheet1 (2).csv")
df

,course_id,text,text_vi,count,true_label
0,C_1011639,各位都是清华的,Mọi người đều đến từ Đại học Thanh Hoa,1.0,3.0
1,C_1011639,倒计时2min,Đếm ngược đến 2 phút,1.0,3.0
2,C_1011639,实在是闲居然很想看这个直播,Tôi thực sự muốn xem chương trình phát sóng tr...,1.0,4.0
3,C_1011639,签到,Đăng nhập,23.0,3.0
4,C_1011639,雨课堂加入不了班级,Không thể tham gia lớp học trong Rain Classroom,1.0,2.0
...,...,...,...,...,...
144782,C_955163,userid没有绑定成功吧,Userid không bị ràng buộc thành công?,1.0,NaN
144783,C_955163,讲的真好,Nói rất hay,1.0,NaN
144784,C_955163,通俗易懂,Dễ hiểu,1.0,NaN
144785,C_955163,easy,dễ,1.0,NaN


# 1. Dữ liệu đã gán nhãn (10000 dòng đầu tiên)

In [2]:
import pandas as pd

# Giả sử dữ liệu của bạn là df
# Chọn 10.000 dòng đầu tiên để huấn luyện
train_df = df.head(10000)

# Kiểm tra các cột dữ liệu
train_df[['text_vi', 'true_label']].head()

,text_vi,true_label
0,Mọi người đều đến từ Đại học Thanh Hoa,3.0
1,Đếm ngược đến 2 phút,3.0
2,Tôi thực sự muốn xem chương trình phát sóng tr...,4.0
3,Đăng nhập,3.0
4,Không thể tham gia lớp học trong Rain Classroom,2.0


In [3]:
train_data = train_df.copy()
train_data

,course_id,text,text_vi,count,true_label
0,C_1011639,各位都是清华的,Mọi người đều đến từ Đại học Thanh Hoa,1.0,3.0
1,C_1011639,倒计时2min,Đếm ngược đến 2 phút,1.0,3.0
2,C_1011639,实在是闲居然很想看这个直播,Tôi thực sự muốn xem chương trình phát sóng tr...,1.0,4.0
3,C_1011639,签到,Đăng nhập,23.0,3.0
4,C_1011639,雨课堂加入不了班级,Không thể tham gia lớp học trong Rain Classroom,1.0,2.0
...,...,...,...,...,...
9995,C_2287011,积极参赛来培养自己的能力,Tích cực tham gia các cuộc thi để phát triển k...,1.0,4.0
9996,C_2287011,手机的屏内指纹骨传导耳机以及各种产品的外观设计这都是最近几年来的创新产品反映了近几年在这几年...,Dấu vân tay trong màn hình của điện thoại di đ...,1.0,3.0
9997,C_2287011,共享充电宝解决了出门手机没电的问题避免手机没电而联系不到人出现危险,Sạc dự phòng dùng chung giải quyết vấn đề hết ...,1.0,3.0
9998,C_2287011,积累经验勇于尝试敢于挑战,"Tích lũy kinh nghiệm, can đảm thử thách và dám...",1.0,4.0


# 2. Gán nhãn dữ liệu còn lại với KNN

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Giả sử 'train_data' là DataFrame chứa dữ liệu đã gán nhãn
# Chuyển dữ liệu thành DataFrame nếu cần
if isinstance(train_data, list):
    train_data = pd.DataFrame(train_data)

# Lấy 10000 dòng đầu tiên làm dữ liệu huấn luyện đã gán nhãn
train_data = train_df.copy()

# Kiểm tra và chuyển đổi cột 'text_vi' thành chuỗi
train_data['text_vi'] = train_data['text_vi'].astype(str)

# Loại bỏ các dòng có giá trị NaN hoặc rỗng trong cột 'text_vi'
train_data = train_data[train_data['text_vi'].str.strip().ne('')]
# Loại bỏ các dòng có giá trị NaN trong cột 'true_label' của dữ liệu huấn luyện
train_data = train_data.dropna(subset=['true_label'])

# Vector hóa văn bản
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(train_data['text_vi'])

# Giả sử 'df' là DataFrame chứa dữ liệu chưa gán nhãn
# Loại bỏ các dòng có giá trị NaN hoặc rỗng trong 'text_vi' của df
unlabeled_data = df.iloc[10000:]
unlabeled_data.loc[:, 'text_vi'] = unlabeled_data['text_vi'].astype(str)
unlabeled_data = unlabeled_data[unlabeled_data['text_vi'].str.strip().ne('')]  # Loại bỏ dòng rỗng hoặc NaN

# Vector hóa dữ liệu chưa gán nhãn
X_test_tfidf = vectorizer.transform(unlabeled_data['text_vi'])

# Huấn luyện KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_tfidf, train_data['true_label'])

# Dự đoán nhãn cho bộ dữ liệu chưa gán nhãn
unlabeled_data['predictions'] = knn.predict(X_test_tfidf)

# Hiển thị kết quả
unlabeled_data[['text_vi', 'predictions']]

,text_vi,predictions
10000,Tư duy đổi mới là rất quan trọng. Hãy chú ý lu...,3.0
10001,Đổi mới phải kết hợp với giáo dục để nâng cao ...,3.0
10002,Thực hành là tiêu chí duy nhất để kiểm nghiệm ...,4.0
10003,Nắm bắt những điều bất ngờ và dám thử thách nó,4.0
10004,Có tính đến sự phát triển,3.0
...,...,...
144782,Userid không bị ràng buộc thành công?,3.0
144783,Nói rất hay,5.0
144784,Dễ hiểu,5.0
144785,dễ,4.0


In [5]:
import pandas as pd

# Giả sử df là bộ dữ liệu ban đầu của bạn, chứa các cột ['course_id', 'text', 'text_vi', 'count', 'true_label']
# Giả sử unlabeled_data chứa các cột ['text_vi', 'predictions']
# Cập nhật df bằng cách thêm cột 'predictions' vào df
df1 = df.copy()  # Đảm bảo không thay đổi bộ dữ liệu gốc

# Sử dụng chỉ số từ unlabeled_data để ghép vào df (các dòng từ 10000 trở đi)
df1.loc[10000:, 'true_label'] = unlabeled_data['predictions'].values

# Kiểm tra kết quả
df1.head()

,course_id,text,text_vi,count,true_label
0,C_1011639,各位都是清华的,Mọi người đều đến từ Đại học Thanh Hoa,1.0,3.0
1,C_1011639,倒计时2min,Đếm ngược đến 2 phút,1.0,3.0
2,C_1011639,实在是闲居然很想看这个直播,Tôi thực sự muốn xem chương trình phát sóng tr...,1.0,4.0
3,C_1011639,签到,Đăng nhập,23.0,3.0
4,C_1011639,雨课堂加入不了班级,Không thể tham gia lớp học trong Rain Classroom,1.0,2.0


In [6]:
df1.tail()

,course_id,text,text_vi,count,true_label
144782,C_955163,userid没有绑定成功吧,Userid không bị ràng buộc thành công?,1.0,3.0
144783,C_955163,讲的真好,Nói rất hay,1.0,5.0
144784,C_955163,通俗易懂,Dễ hiểu,1.0,5.0
144785,C_955163,easy,dễ,1.0,4.0
144786,C_955163,method设置成post还是参数不足postman调试可以,Nếu phương thức được đặt thành đăng hoặc không...,1.0,3.0


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144787 entries, 0 to 144786
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   course_id   144787 non-null  object 
 1   text        144784 non-null  object 
 2   text_vi     144787 non-null  object 
 3   count       144784 non-null  float64
 4   true_label  144786 non-null  float64
dtypes: float64(2), object(3)
memory usage: 5.5+ MB


# 3. Lưu bộ dữ liệu đã gán nhãn

In [8]:
# Lưu DataFrame thành file Excel
df1.to_excel('/kaggle/working/labeled_comments.xlsx', index=False)
# Lưu DataFrame thành file CSV
df1.to_csv('/kaggle/working/labeled_comments.csv', index=False)